In [26]:
import pandas as pd
import numpy as np

reactivePower=pd.read_csv('R3_12_47_1_AMI_residential_phase12_reactive_power.csv',skiprows=8)
realPower=pd.read_csv('R3_12_47_1_AMI_residential_phase12_real_power.csv',skiprows=8)

a=realPower.shape
a
type(a)
a[0]

realPower.head(5)

a=realPower.index

aa=realPower.ix[:,0:1]

type(aa)

type(aa[:1])

Create 15 minute interval of datetime array to retrive from the original data if it has minute by minute interval data

In [31]:
def dateArray15mins(data_array):
    from datetime import datetime, timedelta

    def datetime_range(start, end, delta):
        current = start
        while current < end:
            yield current
            current += delta

    dts = [dt.strftime('%Y-%m-%dT%H:%M:%S') for dt in 
           datetime_range((np.datetime64(data_array[1,0]).astype(datetime)),
                          (np.datetime64(data_array[len(data_array)-1,0]).astype(datetime)),
                          timedelta(minutes=15))]
    return dts

#d=dts[1]
#np.datetime64(d)

In [32]:
def ind15mins(data_array):
    ind=[0]
    ind.extend((np.arange(1,len(data_array),15)).tolist())
    new_array=data_array[ind,:]
    return new_array

In [35]:
newReactiveVoltage=ind15mins(reactiveVoltage)
newRealVoltage=ind15mins(realVoltage)

#d=dateArray15mins(realVdataArray)

In [174]:
rT=pd.DataFrame.transpose(reactivePower)
aa=rT.unstack()

In [251]:
realPower['2013-07-01 00']

# timestamp,tpm0_R3-12-47-1_tm_3_R3-12-47-1_tn_411,tpm0_R3-12-47-1_tm_4_R3-12-47-1_tn_412,tpm0_R3-12-47-1_tm_5_R3-12-47-1_tn_413,tpm0_R3-12-47-1_tm_6_R3-12-47-1_tn_414,tpm0_R3-12-47-1_tm_7_R3-12-47-1_tn_415,tpm0_R3-12-47-1_tm_8_R3-12-47-1_tn_416,tpm0_R3-12-47-1_tm_10_R3-12-47-1_tn_418,tpm1_R3-12-47-1_tm_10_R3-12-47-1_tn_418,tpm2_R3-12-47-1_tm_10_R3-12-47-1_tn_418,tpm0_R3-12-47-1_tm_11_R3-12-47-1_tn_419,...,tpm0_R3-12-47-1_tm_396_R3-12-47-1_tn_804,tpm0_R3-12-47-1_tm_397_R3-12-47-1_tn_805,tpm1_R3-12-47-1_tm_397_R3-12-47-1_tn_805,tpm0_R3-12-47-1_tm_398_R3-12-47-1_tn_806,tpm1_R3-12-47-1_tm_398_R3-12-47-1_tn_806,tpm0_R3-12-47-1_tm_399_R3-12-47-1_tn_807,tpm1_R3-12-47-1_tm_399_R3-12-47-1_tn_807,tpm0_R3-12-47-1_tm_400_R3-12-47-1_tn_808,tpm0_R3-12-47-1_tm_401_R3-12-47-1_tn_809,tpm0_R3-12-47-1_tm_402_R3-12-47-1_tn_810
0,,,,,,,,,,,,,,,,,,,,,
2013-07-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-01 00:15:00,205.339724,205.334689,138.543737,200.350553,131.729988,202.221428,142.890566,307.401462,182.122934,247.544630,...,242.291418,227.750556,252.362746,242.642725,280.125861,243.708437,186.060236,309.673926,252.147617,261.194620
2013-07-01 00:30:00,503.587586,205.329650,138.526331,168.547274,131.725769,202.196619,142.887046,295.194454,182.118396,212.897516,...,242.261576,227.790250,246.610827,242.635082,221.001409,217.072895,186.041403,309.726639,229.971185,255.013213
2013-07-01 00:45:00,486.062104,205.464421,130.265434,168.568571,131.812387,202.081852,142.985282,246.568715,174.515431,212.779126,...,196.512125,208.927300,246.652897,217.770129,221.151961,216.943040,185.930111,309.774297,230.122817,234.665461


In [252]:
realPower.resample('30T').sum()

# timestamp,tpm0_R3-12-47-1_tm_3_R3-12-47-1_tn_411,tpm0_R3-12-47-1_tm_4_R3-12-47-1_tn_412,tpm0_R3-12-47-1_tm_5_R3-12-47-1_tn_413,tpm0_R3-12-47-1_tm_6_R3-12-47-1_tn_414,tpm0_R3-12-47-1_tm_7_R3-12-47-1_tn_415,tpm0_R3-12-47-1_tm_8_R3-12-47-1_tn_416,tpm0_R3-12-47-1_tm_10_R3-12-47-1_tn_418,tpm1_R3-12-47-1_tm_10_R3-12-47-1_tn_418,tpm2_R3-12-47-1_tm_10_R3-12-47-1_tn_418,tpm0_R3-12-47-1_tm_11_R3-12-47-1_tn_419,...,tpm0_R3-12-47-1_tm_396_R3-12-47-1_tn_804,tpm0_R3-12-47-1_tm_397_R3-12-47-1_tn_805,tpm1_R3-12-47-1_tm_397_R3-12-47-1_tn_805,tpm0_R3-12-47-1_tm_398_R3-12-47-1_tn_806,tpm1_R3-12-47-1_tm_398_R3-12-47-1_tn_806,tpm0_R3-12-47-1_tm_399_R3-12-47-1_tn_807,tpm1_R3-12-47-1_tm_399_R3-12-47-1_tn_807,tpm0_R3-12-47-1_tm_400_R3-12-47-1_tn_808,tpm0_R3-12-47-1_tm_401_R3-12-47-1_tn_809,tpm0_R3-12-47-1_tm_402_R3-12-47-1_tn_810
0,,,,,,,,,,,,,,,,,,,,,
2013-07-01 00:00:00,205.339724,205.334689,138.543737,200.350553,131.729988,202.221428,142.890566,307.401462,182.122934,247.544630,...,242.291418,227.750556,252.362746,242.642725,280.125861,243.708437,186.060236,309.673926,252.147617,261.194620
2013-07-01 00:30:00,989.649690,410.794071,268.791765,337.115845,263.538156,404.278471,285.872328,541.763169,356.633827,425.676642,...,438.773701,436.717550,493.263724,460.405211,442.153370,434.015935,371.971514,619.500936,460.094002,489.678674
2013-07-01 01:00:00,923.005648,375.898164,251.087306,331.775881,242.810853,349.712338,276.174391,493.959415,331.917782,350.801321,...,393.103505,410.884841,446.049306,378.956412,441.411213,422.330412,351.775048,530.991349,448.284008,469.407590
2013-07-01 01:30:00,459.366546,184.499893,125.724715,500.111244,119.359589,160.416704,135.204465,236.290499,166.069185,169.044663,...,196.831712,204.754759,194.283414,189.614063,197.292803,195.458704,175.421200,246.034329,209.461800,231.897850


In [267]:
realPower.resample('30T').mean()#,how='mean')

# timestamp,tpm0_R3-12-47-1_tm_3_R3-12-47-1_tn_411,tpm0_R3-12-47-1_tm_4_R3-12-47-1_tn_412,tpm0_R3-12-47-1_tm_5_R3-12-47-1_tn_413,tpm0_R3-12-47-1_tm_6_R3-12-47-1_tn_414,tpm0_R3-12-47-1_tm_7_R3-12-47-1_tn_415,tpm0_R3-12-47-1_tm_8_R3-12-47-1_tn_416,tpm0_R3-12-47-1_tm_10_R3-12-47-1_tn_418,tpm1_R3-12-47-1_tm_10_R3-12-47-1_tn_418,tpm2_R3-12-47-1_tm_10_R3-12-47-1_tn_418,tpm0_R3-12-47-1_tm_11_R3-12-47-1_tn_419,...,tpm0_R3-12-47-1_tm_396_R3-12-47-1_tn_804,tpm0_R3-12-47-1_tm_397_R3-12-47-1_tn_805,tpm1_R3-12-47-1_tm_397_R3-12-47-1_tn_805,tpm0_R3-12-47-1_tm_398_R3-12-47-1_tn_806,tpm1_R3-12-47-1_tm_398_R3-12-47-1_tn_806,tpm0_R3-12-47-1_tm_399_R3-12-47-1_tn_807,tpm1_R3-12-47-1_tm_399_R3-12-47-1_tn_807,tpm0_R3-12-47-1_tm_400_R3-12-47-1_tn_808,tpm0_R3-12-47-1_tm_401_R3-12-47-1_tn_809,tpm0_R3-12-47-1_tm_402_R3-12-47-1_tn_810
0,,,,,,,,,,,,,,,,,,,,,
2013-07-01 00:00:00,102.669862,102.667344,69.271868,100.175276,65.864994,101.110714,71.445283,153.700731,91.061467,123.772315,...,121.145709,113.875278,126.181373,121.321363,140.062930,121.854219,93.030118,154.836963,126.073808,130.597310
2013-07-01 00:30:00,494.824845,205.397035,134.395882,168.557922,131.769078,202.139235,142.936164,270.881585,178.316913,212.838321,...,219.386850,218.358775,246.631862,230.202606,221.076685,217.007968,185.985757,309.750468,230.047001,244.839337
2013-07-01 01:00:00,461.502824,187.949082,125.543653,165.887941,121.405427,174.856169,138.087196,246.979708,165.958891,175.400661,...,196.551753,205.442420,223.024653,189.478206,220.705606,211.165206,175.887524,265.495675,224.142004,234.703795
2013-07-01 01:30:00,459.366546,184.499893,125.724715,500.111244,119.359589,160.416704,135.204465,236.290499,166.069185,169.044663,...,196.831712,204.754759,194.283414,189.614063,197.292803,195.458704,175.421200,246.034329,209.461800,231.897850


In [193]:
import csv
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as pyplot

Function to convert CSV files to pandas DataFrame. Discarding the description sentences on top of each file

In [211]:
def csv2array(fileName):
    newdata=[]
    with open(fileName,'r') as fileData:
        rowData = csv.reader(fileData, 
                           delimiter = ',', 
                           quotechar = '"')
        for data in rowData:
            if len(data)>2:
                newdata.append(data)
                type(data)
    data_array = np.asarray(newdata)
    return convertDateTime(data_array)

Fist Column of the array is datetime with 'PST' extension. This function strips timezone and converts it into numpy datetime64 format

In [288]:
def convertDateTime(data_array):
    firstCol=data_array[1:,0]
    stripFirstCol=[ele.rstrip(' PST') for ele in firstCol]
    dateTimeFirstCol=[np.datetime64(ele) for ele in stripFirstCol]
    data_array[1:,0]=dateTimeFirstCol
    data_frame=pd.DataFrame(data_array.reshape(len(data_array),-1))
    
    data_frame.index=data_frame[0]
    data_frame=data_frame.drop(data_frame.columns[0],axis=1)


    data_frame.columns=data_frame.iloc[0]
    data_frame=data_frame.drop(data_frame.index[0])
    
    data_frame=data_frame.astype(str).astype(float)
    data_frame.index=pd.to_datetime(data_frame.index)
    data_frame=data_frame.resample('15T').mean()
    return data_frame

In [289]:
reactivePfile='R3_12_47_1_AMI_residential_phase12_reactive_power.csv'
realPfile='R3_12_47_1_AMI_residential_phase12_real_power.csv'
reactiveVfile='R3_12_47_1_AMI_residential_phase12_reactive_voltage.csv'
realVfile='R3_12_47_1_AMI_residential_phase12_real_voltage.csv'

In [290]:
reactivePower=csv2array(reactivePfile)
realPower=csv2array(realPfile)

reactiveVoltage=csv2array(reactiveVfile)
realVoltage=csv2array(realVfile)

In [298]:
apparentVoltage=((realVoltage**2)+(reactiveVoltage**2))**0.5
apparentPower=((realPower**2)+(reactivePower**2))**0.5
independentVariable=apparentVoltage/240